In [197]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import calendar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
cd D:\Course\SBRS

D:\Course\SBRS


In [199]:
ctg_file = 'datasets/retailrocket/category_tree.csv'
events_file = 'datasets/retailrocket/events.csv'
item_prop_1 = 'datasets/retailrocket/item_properties_part1.csv'
item_prop_2 = 'datasets/retailrocket/item_properties_part2.csv'

In [200]:
ctg_df = pd.read_csv(ctg_file)
events_df = pd.read_csv(events_file)
item_prop_1_df = pd.read_csv(item_prop_1)
item_prop_2_df = pd.read_csv(item_prop_2)


In [201]:
item_prop_1_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [202]:
ctg_df.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [203]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [204]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [205]:
events_actions = {
    'view': 1,
    'addtocart':2,
    'transaction':3
}

In [206]:
events_df.event = [events_actions[item] for item in events_df.event]

In [207]:
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,1,355908,NaN
1,1433224214164,992329,1,248676,NaN
2,1433221999827,111016,1,318965,NaN
3,1433221955914,483717,1,253185,NaN
4,1433221337106,951259,1,367447,NaN


In [208]:
events_df['event_date'] = pd.to_datetime(events_df['timestamp'])
events_df = events_df.drop('timestamp', axis=1)


In [209]:
events_df['date'] = events_df.event_date.dt.date
events_df['time'] = events_df.event_date.dt.time
events_df['year'] = events_df.event_date.dt.year
events_df['month'] = events_df.event_date.dt.month
events_df['day'] = events_df.event_date.dt.day
events_df['hour'] = events_df.event_date.dt.hour
events_df = events_df.drop(['event_date','time','date'], axis=1)

In [210]:
events_df.head()

,visitorid,event,itemid,transactionid,year,month,day,hour
0,257597,1,355908,NaN,1970,1,1,0
1,992329,1,248676,NaN,1970,1,1,0
2,111016,1,318965,NaN,1970,1,1,0
3,483717,1,253185,NaN,1970,1,1,0
4,951259,1,367447,NaN,1970,1,1,0


In [211]:
events_df.transactionid.fillna(-1, inplace=True)

In [212]:
events_df.head()

,visitorid,event,itemid,transactionid,year,month,day,hour
0,257597,1,355908,-1.0,1970,1,1,0
1,992329,1,248676,-1.0,1970,1,1,0
2,111016,1,318965,-1.0,1970,1,1,0
3,483717,1,253185,-1.0,1970,1,1,0
4,951259,1,367447,-1.0,1970,1,1,0


In [213]:
len(events_df.visitorid.unique())

1407580

In [214]:
vis_event_df = events_df[['visitorid', 'event', 'itemid']].sort_values(['visitorid', 'event'])
vis_event_df.head(30)

,visitorid,event,itemid
1361687,0,1,285930
1367212,0,1,357564
1367342,0,1,67045
830385,1,1,72028
726292,2,1,216305
728288,2,1,325215
735202,2,1,342816
735273,2,1,325215
737615,2,1,342816
737711,2,1,259884


In [215]:
events_df_new = events_df[['visitorid', 'event', 'itemid']]
events_df_new.head()

,visitorid,event,itemid
0,257597,1,355908
1,992329,1,248676
2,111016,1,318965
3,483717,1,253185
4,951259,1,367447


In [216]:
items_list = events_df.itemid.unique()
items_list

array([355908, 248676, 318965, ...,    613,  52086, 177353], dtype=int64)

In [217]:
user_item_matrix = events_df_new.pivot_table(index='visitorid', columns='itemid', values='event')
user_item_matrix.head()

D:\Course\SBRS\.venv\lib\site-packages\pandas\core\reshape\reshape.py:126: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


IndexError: index 154794359 is out of bounds for axis 0 with size 154680588